# Немного теории
https://www.geeksforgeeks.org/transaction-isolation-levels-dbms/

https://en.wikipedia.org/wiki/ACID


# Создаем базу данные sqllight в оперативной памяти


In [1]:
# Импорт необходимых библиотек
import sqlite3
import pandas as pd

# Подключение к базе данных (создание базы данных в оперативной памяти)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Создание таблицы superheroes
cursor.execute('''
    CREATE TABLE superheroes (
        id INTEGER PRIMARY KEY,
        name TEXT,
        align TEXT,
        eye TEXT,
        hair TEXT,
        gender TEXT,
        appearances INTEGER,
        year INTEGER,
        universe TEXT
    )
''')

# Вставка данных в таблицу
superheroes_data = [
    (1, 'Spider-Man', 'good', 'hazel', 'brown', 'Male', 4043, 1962, 'marvel'),
    (2, 'Spider-Man-2', 'good', 'hazel', 'brown', 'Male', 4000, 1900, 'marvel'),
    (3, 'Spider-Man-3', 'good', 'hazel', 'brown', 'Male', 3000, 1800, 'marvel'),
    (4, 'Batman', 'good', 'blue', 'black', 'Male', 3389, 1939, 'dc'),
    (5, 'Wonder Woman', 'good', 'blue', 'black', 'Female', 1023, 1941, 'dc'),
    (6, 'Iron Man', 'good', 'blue', 'black', 'Male', 3327, 1963, 'marvel'),
    (7, 'Iron Man-2', 'bad', 'blue', 'black', 'Male', 3327, 1963, 'marvel'),
    (8, 'Joker', 'bad', 'green', 'green', 'Male', 1232, 1940, 'dc')
]

cursor.executemany('''
    INSERT INTO superheroes (id, name, align, eye, hair, gender, appearances, year, universe)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', superheroes_data)
conn.commit()

# Отображение всех данных таблицы superheroes
df = pd.read_sql('SELECT * FROM superheroes', conn)
df


,id,name,align,eye,hair,gender,appearances,year,universe
0,1,Spider-Man,good,hazel,brown,Male,4043,1962,marvel
1,2,Spider-Man-2,good,hazel,brown,Male,4000,1900,marvel
2,3,Spider-Man-3,good,hazel,brown,Male,3000,1800,marvel
3,4,Batman,good,blue,black,Male,3389,1939,dc
4,5,Wonder Woman,good,blue,black,Female,1023,1941,dc
5,6,Iron Man,good,blue,black,Male,3327,1963,marvel
6,7,Iron Man-2,bad,blue,black,Male,3327,1963,marvel
7,8,Joker,bad,green,green,Male,1232,1940,dc


 # Основы SELECT и WHERE

In [4]:
# Выбор всех записей из таблицы superheroes
df_all = pd.read_sql('SELECT * FROM superheroes', conn)
df_all

,id,name,align,eye,hair,gender,appearances,year,universe
0,1,Spider-Man,good,hazel,brown,Male,4043,1962,marvel
1,2,Spider-Man-2,good,hazel,brown,Male,4000,1900,marvel
2,3,Spider-Man-3,good,hazel,brown,Male,3000,1800,marvel
3,4,Batman,good,blue,black,Male,3389,1939,dc
4,5,Wonder Woman,good,blue,black,Female,1023,1941,dc
5,6,Iron Man,good,blue,black,Male,3327,1963,marvel
6,7,Iron Man-2,bad,blue,black,Male,3327,1963,marvel
7,8,Joker,bad,green,green,Male,1232,1940,dc


In [5]:
# Выбор записей, где "align" (сторона) - "good"
# year BETWEEN 2000 AND 2010 -  also possible, like '%d'
df_good = pd.read_sql("SELECT * FROM superheroes WHERE align = 'good'", conn)
df_good

,id,name,align,eye,hair,gender,appearances,year,universe
0,1,Spider-Man,good,hazel,brown,Male,4043,1962,marvel
1,2,Spider-Man-2,good,hazel,brown,Male,4000,1900,marvel
2,3,Spider-Man-3,good,hazel,brown,Male,3000,1800,marvel
3,4,Batman,good,blue,black,Male,3389,1939,dc
4,5,Wonder Woman,good,blue,black,Female,1023,1941,dc
5,6,Iron Man,good,blue,black,Male,3327,1963,marvel


In [10]:
# Выбор всех записей из таблицы superheroes
df_all = pd.read_sql('SELECT * FROM superheroes LIMIT 2', conn)
df_all

,id,name,align,eye,hair,gender,appearances,year,universe
0,1,Spider-Man,good,hazel,brown,Male,4043,1962,marvel
1,2,Spider-Man-2,good,hazel,brown,Male,4000,1900,marvel


In [11]:
# Выбор всех записей из таблицы superheroes
df_all = pd.read_sql('SELECT * FROM superheroes ORDER BY year desc LIMIT 5 ', conn)
df_all

,id,name,align,eye,hair,gender,appearances,year,universe
0,6,Iron Man,good,blue,black,Male,3327,1963,marvel
1,7,Iron Man-2,bad,blue,black,Male,3327,1963,marvel
2,1,Spider-Man,good,hazel,brown,Male,4043,1962,marvel
3,5,Wonder Woman,good,blue,black,Female,1023,1941,dc
4,8,Joker,bad,green,green,Male,1232,1940,dc


# Агрегатные функции

In [14]:
# Подсчет количества героев по вселенной
# distinct оператора
df_count = pd.read_sql('SELECT universe, COUNT(universe) as count FROM superheroes GROUP BY universe', conn)
df_count

,universe,count
0,dc,3
1,marvel,5


In [15]:
# Подсчет количества героев по вселенной и фильтр тех кого больше или равно трем
df_count_filter = pd.read_sql('''
    SELECT universe, COUNT(*) as count
    FROM superheroes
    GROUP BY universe
    HAVING count >= 4
''', conn)
df_count_filter

,universe,count
0,dc,3
1,marvel,5


In [19]:
# Привет с условием where align is good,
df_where = pd.read_sql('''
    SELECT universe, COUNT(*) as count
    FROM superheroes
    WHERE align = 'good'
    GROUP BY universe
    HAVING count > 3
''', conn)
df_where

,universe,count
0,marvel,4


# Использование оконных функций

In [4]:
df_row_number = pd.read_sql('''
    SELECT
        ROW_NUMBER() OVER (ORDER BY appearances DESC) AS position,
        name,
        appearances
    FROM superheroes
''', conn)
df_row_number

,position,name,appearances
0,1,Spider-Man,4043
1,2,Batman,3389
2,3,Iron Man,3327
3,4,Joker,1232
4,5,Wonder Woman,1023


# Функции смещения

In [39]:
# Применение функции LAG для получения предыдущего числа появлений
# LAG(column_name, offset, default_value)
df_lag = pd.read_sql('''
    SELECT
        name,
        appearances,
        LAG(appearances, 1, 0) OVER (ORDER BY appearances) AS previous_appearances
    FROM superheroes
''', conn)
df_lag


,name,appearances,previous_appearances
0,Wonder Woman,1023,0
1,Joker,1232,1023
2,Spider-Man-3,3000,1232
3,Iron Man,3327,3000
4,Iron Man-2,3327,3327
5,Batman,3389,3327
6,Spider-Man-2,4000,3389
7,Spider-Man,4043,4000


 # Использование PARTITION BY и функции RANK:

In [40]:
# Пример использования PARTITION BY и функции RANK
df_partition_rank = pd.read_sql('''
    SELECT
        universe,
        name,
        appearances,
        RANK() OVER (PARTITION BY universe ORDER BY appearances DESC) AS rank
    FROM superheroes
''', conn)
df_partition_rank


,universe,name,appearances,rank
0,dc,Batman,3389,1
1,dc,Joker,1232,2
2,dc,Wonder Woman,1023,3
3,marvel,Spider-Man,4043,1
4,marvel,Spider-Man-2,4000,2
5,marvel,Iron Man,3327,3
6,marvel,Iron Man-2,3327,3
7,marvel,Spider-Man-3,3000,5


In [3]:
df_tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
df_tables

,name
0,universes
1,superheroes


In [2]:
# Создание таблицы universes
cursor.execute('''
    CREATE TABLE universes (
        id INTEGER PRIMARY KEY,
        name TEXT
    )
''')

# Вставка данных в таблицу universes
universes_data = [
    (1, 'marvel'),
    (2, 'dc')
]

cursor.executemany('''
    INSERT INTO universes (id, name)
    VALUES (?, ?)
''', universes_data)

# Обновление таблицы superheroes для добавления внешнего ключа
cursor.execute('''
    ALTER TABLE superheroes ADD COLUMN universe_id INTEGER
''')

# Связка данных между superheroes и universes
cursor.execute('''
    UPDATE superheroes
    SET universe_id = (
        SELECT id FROM universes WHERE universes.name = superheroes.universe
    )
''')

# Удаление старого столбца universe (уже не нужен после добавления внешнего ключа)
cursor.execute('''
    CREATE TABLE superheroes_new AS
    SELECT id, name, align, eye, hair, gender, appearances, year, universe_id
    FROM superheroes
''')

cursor.execute('DROP TABLE superheroes')
cursor.execute('ALTER TABLE superheroes_new RENAME TO superheroes')

conn.commit()


In [4]:
df_partition_joined = pd.read_sql('''SELECT superheroes.name, universes.name AS universe_name
FROM superheroes
JOIN universes ON superheroes.universe_id = universes.id;
''', conn);

df_partition_joined

,name,universe_name
0,Spider-Man,marvel
1,Spider-Man-2,marvel
2,Spider-Man-3,marvel
3,Batman,dc
4,Wonder Woman,dc
5,Iron Man,marvel
6,Iron Man-2,marvel
7,Joker,dc


In [35]:
# Без особой надобности лучше так не делать, но так делать можно ;)
df_partition_joined = pd.read_sql('''SELECT superheroes.name AS hero_name, universes.name AS universe_name
FROM superheroes
CROSS JOIN universes;
''', conn);
df_partition_joined


,hero_name,universe_name
0,Spider-Man,marvel
1,Spider-Man,dc
2,Spider-Man-2,marvel
3,Spider-Man-2,dc
4,Spider-Man-3,marvel
5,Spider-Man-3,dc
6,Batman,marvel
7,Batman,dc
8,Wonder Woman,marvel
9,Wonder Woman,dc


In [6]:
# Добавление данных с пустыми значениями
superheroes_data_with_nulls = [
    (9, 'Invisible Man', 'neutral', None, 'blonde', 'Male', 0, None, 1),  # Всего 1 пустое значение
    (10, 'Unknown Hero', None, None, None, None, None, None, 2),  # Все значения пустые кроме universe_id
    (11, 'Half Data Hero', 'good', 'blue', None, 'Female', 234, None, 1),  # Отсутствует год
]

cursor.executemany('''
    INSERT INTO superheroes (id, name, align, eye, hair, gender, appearances, year, universe_id)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', superheroes_data_with_nulls)

conn.commit()

# Пример запроса, который выбирает первый ненулевой из двух столбцов
# Например, выбираем первый ненулевой между столбцами 'eye' и 'hair'

df_with_coalesce = pd.read_sql('''
    SELECT id, name,
           COALESCE(eye, hair) AS first_non_null_feature
    FROM superheroes
''', conn)
df_with_coalesce


,id,name,first_non_null_feature
0,1,Spider-Man,hazel
1,2,Spider-Man-2,hazel
2,3,Spider-Man-3,hazel
3,4,Batman,blue
4,5,Wonder Woman,blue
5,6,Iron Man,blue
6,7,Iron Man-2,blue
7,8,Joker,green
8,9,Invisible Man,blonde
9,10,Unknown Hero,None


In [26]:
conn.close()